# Data Wrangling paso a paso (caso urgencias respiratorias)

**Objetivo del notebook**  
Mostrar, de forma didáctica y reproducible, cómo transformar un dataset real en uno confiable y analizable mediante técnicas de **data wrangling**.

El foco no está en el análisis final, sino en **las decisiones intermedias**.

## 1. Contexto del dataset

Trabajaremos con un archivo de atenciones de urgencia respiratoria en la Región Metropolitana, que incluye:
- establecimientos de salud,
- variables territoriales,
- causas clínicas,
- conteos por tramo etario,
- información temporal (año / semana).

Este tipo de datos **no viene listo para análisis**: mezcla variables clínicas, administrativas y operativas.

## 2. Cargar datos

In [3]:
import pandas as pd

path = "data_stgo.csv" 
df_raw = pd.read_csv(path)

df_raw.shape

(34325, 32)

## 3. Inspección inicial

Antes de tocar los datos, hay que mirarlos: estructura, primeras filas y tipos.

In [4]:
df_raw.head()

# Tipos de datos
df_raw.dtypes

Unnamed: 0                     int64
EstablecimientoCodigo          int64
EstablecimientoGlosa          object
RegionCodigo                   int64
RegionGlosa                   object
ComunaCodigo                   int64
ComunaGlosa                   object
ServicioSaludCodigo            int64
ServicioSaludGlosa            object
TipoEstablecimiento           object
DependenciaAdministrativa     object
NivelAtencion                 object
TipoUrgencia                  object
Latitud                      float64
Longitud                     float64
NivelComplejidad              object
Anio                           int64
SemanaEstadistica              int64
OrdenCausa                     int64
Causa                         object
NumTotal                     float64
NumMenor1Anio                float64
Num1a4Anios                  float64
Num5a14Anios                 float64
Num15a64Anios                float64
Num65oMas                    float64
TipoUrgencia_std              object
T

## 4. Limpieza estructural básica

### 4.1 Eliminar columnas irrelevantes

`Unnamed: 0` suele ser un índice antiguo guardado al exportar a CSV.  
No es parte del fenómeno que estamos estudiando.

In [5]:
df = df_raw.drop(columns=["Unnamed: 0"], errors="ignore")

df.shape

(34325, 31)

## 5. Conversión y revisión de tipos de datos

La idea es que las columnas se comporten como lo que son (número, texto, categoría).  
Los conteos deberían ser numéricos y aceptar nulos.

In [6]:
df.dtypes

EstablecimientoCodigo          int64
EstablecimientoGlosa          object
RegionCodigo                   int64
RegionGlosa                   object
ComunaCodigo                   int64
ComunaGlosa                   object
ServicioSaludCodigo            int64
ServicioSaludGlosa            object
TipoEstablecimiento           object
DependenciaAdministrativa     object
NivelAtencion                 object
TipoUrgencia                  object
Latitud                      float64
Longitud                     float64
NivelComplejidad              object
Anio                           int64
SemanaEstadistica              int64
OrdenCausa                     int64
Causa                         object
NumTotal                     float64
NumMenor1Anio                float64
Num1a4Anios                  float64
Num5a14Anios                 float64
Num15a64Anios                float64
Num65oMas                    float64
TipoUrgencia_std              object
TipoUrgencia_cat              object
T

### 5.1 Conteos numéricos (nullable)

Usamos `Int64` (con I mayúscula) porque permite valores faltantes (`NA`).

In [7]:
conteo_cols = [
    "NumTotal","NumMenor1Anio","Num1a4Anios",
    "Num5a14Anios","Num15a64Anios","Num65oMas"
]

# Convertir a Int64 si es posible
for c in conteo_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

df[conteo_cols].dtypes

NumTotal         Int64
NumMenor1Anio    Int64
Num1a4Anios      Int64
Num5a14Anios     Int64
Num15a64Anios    Int64
Num65oMas        Int64
dtype: object

## 6. Estandarización de categorías

La estandarización no busca “borrar” el original, sino crear una versión **comparable** para análisis.

### 6.1 Tipo de urgencia

In [8]:
# Conserva el original y crea una versión categórica
urg_map = {
    "URGENCIA HOSPITALARIA (UEH)": "HOSPITALARIA",
    "URGENCIA AMBULATORIA (SAPU)": "AMBULATORIA",
    "URGENCIA ESPECIALIZADA": "ESPECIALIZADA",
}

df["TipoUrgencia_cat"] = (
    df["TipoUrgencia"].astype("string").str.upper().str.strip().map(urg_map)
)

df[["TipoUrgencia", "TipoUrgencia_cat"]].drop_duplicates()

,TipoUrgencia,TipoUrgencia_cat
0,Urgencia Hospitalaria (UEH),HOSPITALARIA
2,Urgencia Ambulatoria (SAPU),AMBULATORIA
7,Urgencia Especializada,ESPECIALIZADA


### 6.2 Tipo de establecimiento

In [9]:
estab_map = {
    "HOSPITAL": "HOSPITAL",
    "SERVICIO DE ATENCIÓN PRIMARIA DE URGENCIA (SAPU)": "SAPU",
}

df["TipoEstablecimiento_cat"] = (
    df["TipoEstablecimiento"].astype("string").str.upper().str.strip().map(estab_map)
)

df[["TipoEstablecimiento", "TipoEstablecimiento_cat"]].drop_duplicates()

,TipoEstablecimiento,TipoEstablecimiento_cat
0,Hospital,HOSPITAL
2,Servicio de Atención Primaria de Urgencia (SAPU),SAPU


## 7. Corrección de valores erróneos (con flags)

Aquí la idea es **marcar** problemas antes de decidir si se corrigen o se excluyen.

### 7.1 Coordenadas fuera de rango (aprox. Chile)

No inventamos coordenadas. Solo marcamos lo que está fuera de un rango razonable.

In [10]:
# Asegurar que sean numéricas (por si vienen como texto)
for c in ["Latitud", "Longitud"]:
    if c in df.columns:
        df[c] = (
            df[c].astype("string")
                .str.strip()
                .str.replace(",", ".", regex=False)
        )
        df[c] = pd.to_numeric(df[c], errors="coerce")

lat_ok = df["Latitud"].between(-56, -17)
lon_ok = df["Longitud"].between(-76, -66)

# Distinguimos faltante vs fuera de rango
df["flag_coord_faltante"] = df["Latitud"].isna() | df["Longitud"].isna()
df["flag_coord_fuera_rango"] = (
    df["Latitud"].notna() & df["Longitud"].notna() &
    (~lat_ok | ~lon_ok)
)

(df[["flag_coord_faltante","flag_coord_fuera_rango"]].sum())

flag_coord_faltante       0
flag_coord_fuera_rango    0
dtype: Int64

### 7.2 Conteos negativos

In [11]:
df["flag_conteo_negativo"] = (df[conteo_cols] < 0).any(axis=1)

df["flag_conteo_negativo"].value_counts(dropna=False)

flag_conteo_negativo
False    34325
Name: count, dtype: Int64

## 8. Reglas lógicas: coherencia interna

Un error frecuente en datos administrativos es que el total no cuadre con el detalle.

### 8.1 Total vs suma de tramos etarios

In [12]:
edad_cols = [
    "NumMenor1Anio","Num1a4Anios",
    "Num5a14Anios","Num15a64Anios","Num65oMas"
]

df["suma_tramos"] = df[edad_cols].sum(axis=1, skipna=True)
df["flag_inconsistencia_total"] = df["NumTotal"] != df["suma_tramos"]

df["flag_inconsistencia_total"].value_counts(dropna=False)

flag_inconsistencia_total
False    34325
Name: count, dtype: Int64

## 9. Separar tipo de registro

En la columna `Causa` pueden venir:
- causas clínicas,
- registros operacionales (hospitalizaciones),
- totales agregados.

Separarlos evita doble conteo y mejora la interpretabilidad.

In [13]:
df["TipoRegistro"] = "CLINICO"

# Nota: .str.contains maneja NA con na=False
causa_upper = df["Causa"].astype("string").str.upper()

df.loc[causa_upper.str.contains("TOTAL", na=False), "TipoRegistro"] = "TOTAL"
df.loc[causa_upper.str.contains("HOSPITALIZACIONES", na=False), "TipoRegistro"] = "OPERACIONAL"

df[["Causa","TipoRegistro"]].drop_duplicates().head(20)

,Causa,TipoRegistro
0,Influenza (J09-J11),CLINICO
1,"Covid-19, Virus identificado U07.1",CLINICO
4,"Otra causa respiratoria (J22, J30-J39, J47, J6...",CLINICO
7,Neumonía (J12-J18),CLINICO
9,"HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO...",OPERACIONAL
11,TOTAL CAUSAS SISTEMA RESPIRATORIO,TOTAL
14,Crisis obstructiva bronquial (J40-J46),CLINICO
18,IRA Alta (J00-J06),CLINICO
23,"HOSPITALIZACIONES COVID-19, VIRUS NO IDENTIFIC...",OPERACIONAL
26,"Covid-19, Virus no identificado U07.2",CLINICO


## 10. Dataset preparado para análisis

Wrangling no es analizar: es decidir qué datos son confiables para analizar.

Ejemplo de filtro conservador:
- quedarnos con registros clínicos
- excluir inconsistencias lógicas del total

In [14]:
df_wrangle = df[
    (df["TipoRegistro"] == "CLINICO") &
    (~df["flag_inconsistencia_total"])
].copy()

df_wrangle.shape

(23898, 36)

## 11. Cierre conceptual

Este notebook muestra que el data wrangling:
- es un proceso iterativo,
- requiere criterio (no solo código),
- deja trazabilidad (flags y columnas derivadas),
- y es la base de cualquier análisis serio.

El resultado no es solo un dataset "limpio", sino un dataset **defendible**.